In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [12]:
import os

from tqdm import progress_bar
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from results import invoke

In [57]:
quantity = 50
path_to_data = f"{os.environ.get('url')}/templates?&state=approved&_sort=updatedAt:DESC&_limit={quantity}"
df = pd.DataFrame(pd.read_json(path_to_data)['uploadedImage'].values.tolist())
urls = list(df['url'])

In [58]:
efn_results = {"imgs": [], "preds": []}
ffd_results = {"imgs": [], "preds": []}
for i, url in tqdm(enumerate(urls)):
    img, preds = invoke("dev-efficientnet-2", url)
    efn_results["imgs"].append(img)
    efn_results["preds"].append(preds)
    img, preds = invoke("dev-facial-features-detector", url)
    ffd_results["imgs"].append(img)
    ffd_results["preds"].append(preds)

50it [02:44,  3.29s/it]


In [61]:
fig, axes = plt.subplots(nrows=len(urls), ncols=2, figsize=(20,300))
plt.tight_layout()
for ax, col in zip(axes[0], ["dev-efficientnet-2", "facial-features-detector"]):
    ax.set_title(col)
for i, url in tqdm(enumerate(urls)):
    axes[i, 0].imshow(np.array(efn_results['imgs'][i]))
    axes[i, 0].scatter(np.array(efn_results['preds'][i][0::2]), np.array(efn_results['preds'][i][1::2]))
    axes[i, 1].imshow(np.array(ffd_results['imgs'][i]))
    axes[i, 1].scatter(np.array(ffd_results['preds'][i][1::2]), np.array(ffd_results['preds'][i][0::2]))